In [4]:
import re
import csv

In [5]:
#Open and read the file
with open("chess_tournament.txt") as f:
    lines = f.read().splitlines()

#Remove empty lines first
lines = [line for line in lines if line]

#Then remove lines that are only made of dashes
lines = [line for line in lines if set(line) != {"-"}]

In [6]:
#Find the starting point where player records begin
record_start = next(i for i, line in enumerate(lines) if re.match(r'\s*\d+', line))

#Group the lines into pairs: main record and details for each player
records = [(lines[i], lines[i+1]) for i in range(record_start, len(lines), 2)]

In [7]:
#Create a dictionary to hold all player data
players = {}

#Go through each record (main line and detail line)
for main, detail in records:
    main_parts = [part.strip() for part in main.split('|')]
    detail_parts = [part.strip() for part in detail.split('|')]

    #First part is the player's number
    player_number = int(main_parts[0])  
    name = main_parts[1]
    
    #Ensure total_points is treated as a numeric value (either int or float based on format)
    total_points = main_parts[2]
    try:
        total_points = float(total_points)  #or int(total_points) if it's an integer
    except ValueError:
        total_points = 0  #Default to 0 if conversion fails

    state = detail_parts[0]

    #Get rating from something like "R: 1794"
    rating_match = re.search(r'R:\s*(\d+)', detail_parts[1])
    rating = int(rating_match.group(1)) if rating_match else 0  # Default to 0 if no match found
    
    #Get opponent numbers from the rest of the main line
    opponents = []
    for part in main_parts[3:]:
        opponents += [int(n) for n in re.findall(r'\d+', part)]

    #Save the player info
    players[player_number] = {
        "Name": name,
        "State": state,
        "Total": total_points,
        "Rating": rating,
        "Opps": opponents
    }

In [8]:
#Calculate average opponent rating
for num, player in players.items():
    opponent_ratings = []
    
    for opp_num in player["Opps"]:
        opponent_ratings.append(players[opp_num]["Rating"])

    avg_opp = round(sum(opponent_ratings) / len(opponent_ratings)) if opponent_ratings else 0
    player["AvgOpp"] = avg_opp

In [9]:
#Define what columns to include in the CSV
fieldnames = ["Name", "State", "Total", "Rating", "AvgOpp"]

#Write everything to a CSV file
with open("chess_tournament_results.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for player_num in sorted(players):
        row = {field: players[player_num][field] for field in fieldnames}
        writer.writerow(row)

print("CSV file 'chess_tournament_results.csv' has been created.")

CSV file 'chess_tournament_results.csv' has been created.
